In [ ]:
# Notebook to analyze vibration measurements. 
#
# Measurement setup: 
#   laser --> cavity fiber --> cavity --> free-space objective --> photo-diode --> yokogawa ossiloscope
#
# This Notebook is old and only exists to store previously done analysis. 
#
# Last and documented version of Wouter is oct2017 version.
#
# QuTech/Hansonlab, Wouter Westerveld, September 2017
#

import numpy as np
import os.path
import pandas
import matplotlib
import scipy.signal
import scipy.constants

%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import gridspec

import mpld3
from mpld3 import plugins

# data_dir = r"K:\ns\qt\Diamond\Projects\Cavities\Cavity characterisation paper\data\data_for_cav_char_paper\20160406\yoko_scans"
# data_dir = '/Users/wjwesterveld/Documents/Temp/' # copied above data to local temp folder, also to be sure we dont overwrite anyting
data_dir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170623/Scope' # copied above data to local temp folder, also to be sure we dont overwrite anyting




## Load xviewer csv (from binary scope data) and scope csv

In [ ]:
def read_scope_xviewer_csv(filename):
    headerRows = 10        
    with open(filename) as f:
        for x in range(headerRows):
            line = f.readline()            
            if "HResolution" in line:                
                timeResolution = float( line[line.find(',')+1:line.rfind(',')].strip(' \t\n\r') )        
    data = pandas.read_csv(filename,header=None,skiprows=headerRows,usecols=[1]).values.flatten()
    return data, timeResolution


def read_scope_csv(filename):
    headerRows = 16        
    with open(filename) as f:
        for x in range(headerRows):
            line = f.readline()            
            if "HResolution" in line:                
                timeResolution = float( line[line.rfind(',')+1:-1].strip(' \t\n\r') )    
    data = pandas.read_csv(filename,header=None,skiprows=headerRows).values[:,1:-1]
    return data, timeResolution



file_name = "IFM_OFF007.csv"  
filename = os.path.join(data_dir,file_name)

data, timeResolution = read_scope_csv(filename)
time = timeResolution * np.arange(data.shape[0]) 

print '{} traces, time resolution {} us (sampling frequency {} kHz), trace length {} s, data length {}'.format( \
    data.shape[1], timeResolution * 1e6, 1.0/timeResolution* 1e-3, timeResolution*len(data), data.shape[0] )


## Plot and Fit linewidth functions

In [ ]:
## Plot functions for linewidth measurements

def tmp_plot_function(time,data):
    fig = plt.figure(figsize=(16, 8))
    plt.subplot(121)
    plt.plot( time, data[:,1] )
    plt.plot( time, data[:,0])
    plt.xlabel('time (s)')
    plt.ylabel('signal (V)')    

    plt.subplot(122)    
    plt.plot( data[:,1], data[:,0])
    plt.xlabel('piezo signal (V)')
    plt.ylabel('cavity transmission (V)')    

    plt.tight_layout()    
    plt.show()

def tmp_read_and_plot(file_name,data_dir):    
    data, timeResolution = read_scope_csv(os.path.join(data_dir,file_name))
    time = timeResolution * np.arange(data.shape[0]) 
    print file_name
    tmp_plot_function(time,data)

In [ ]:
## LINEWIDTH IN GHz - FITTING AND ANALYSIS FUNCTIONS

import pylab
from scipy.optimize import leastsq # Levenberg-Marquadt Algorithm #

def lorentzian(x,p):    
    # Lorenzian function p[0] is center, p[1] is fwhm, p[2] is maximum intensity
    #    See http://mathworld.wolfram.com/LorentzianFunction.html
    #    But note that we not integrated intensity of one for p[2] = 1 but maxiumum intensity.
    return p[2] * (0.5 * p[1])**2 / ( ( x - p[0] )**2 + ( 0.5 * p[1] )**2 )

def lorentzian3(x,p):  
    return p[2] * ( lorentzian(x,[p[0], p[1], 1]) + lorentzian(x,p[3:6]) + lorentzian(x,p[6:9]) )

def fit_lorenzian(time,data,fwhmGuess,timeResolution):
    # initial guess
    a1Guess = np.amax(data)
    t1Guess = np.argmax(data) * timeResolution + time[0]
    # fit lorenzian function to data
    def residuals(p,y,x):
        return y - lorentzian(x,p)
    guessParameters = [t1Guess,fwhmGuess,a1Guess]  # [peak center, hwhm, intensity]
    pbest = leastsq(residuals,guessParameters,args=(data,time),full_output=1)
    fitParameters = pbest[0]
    return fitParameters, guessParameters

def fit_lorenzian3(time,data,fwhmGuess,timeDeltaGuess,timeResolution):
    # initial guess
    a1Guess = np.amax(data) * 0.7
    a2Guess = 1
    a3Guess = 1
    t1Guess = np.argmax(data) * timeResolution + time[0]
    t2Guess = t1Guess + timeDeltaGuess
    t3Guess = t1Guess - timeDeltaGuess
    # Fit three lorenzians to data.
    def residuals(p,y,x):
        return y - lorentzian3(x,p)
    guessParameters = np.array([[t1Guess, fwhmGuess, a1Guess], [t2Guess, fwhmGuess, a2Guess], [t3Guess, fwhmGuess, a3Guess]]).flatten()
    pbest = leastsq(residuals,guessParameters,args=(data,time),full_output=1)
    fitParameters = pbest[0]
    return fitParameters, guessParameters
    


def fit_linewidth_GHz(filename1, time1, time2, filename2, time3, time4, fwhmGuess, timeDeltaGuess):
    ## Fit linewith
    # Load data
    data, timeResolution = read_scope_csv(filename1)
    time = timeResolution * np.arange(data.shape[0]) 
    i1 = int( time1 / timeResolution )
    i2 = int( time2 / timeResolution )
    data = data[i1:i2,0]
    time = time[i1:i2]
    # Fit
    transmissionFitParameters, transmissionGuessParameters = fit_lorenzian(time,data,fwhmGuess,timeResolution)
    # Plot
    fig = plt.figure(figsize=(16, 8))
    plt.subplot(121)    
    plt.plot( time, data )
    plt.plot( time, lorentzian(time,transmissionGuessParameters), linestyle=':' )
    plt.plot( time, lorentzian(time,transmissionFitParameters) )    
    print 'peak position {} s, fwhm {} s, intensity {} V'.format( \
        transmissionFitParameters[0], transmissionFitParameters[1], transmissionFitParameters[2] )

    ## Fit scanning speed with modulated signal 6 GHz
    # Load data
    data, timeResolution = read_scope_csv(filename2)
    time = timeResolution * np.arange(data.shape[0]) 
    i1 = int( time3 / timeResolution )
    i2 = int( time4 / timeResolution )
    data = data[i1:i2,0]
    time = time[i1:i2]
    # Fit
    fitParameters, guessParameters = fit_lorenzian3(time,data,fwhmGuess,timeDeltaGuess,timeResolution)
    # Plot        
    plt.subplot(122)
    plt.plot( time, data )
    plt.plot( time, lorentzian3(time,guessParameters), linestyle=':' )
    plt.plot( time, lorentzian3(time,fitParameters) )
    plt.show()
        
    print 'peak positions (s)', fitParameters[[0,3,6]]
    print 'peak fwhm (s)', fitParameters[[1,4,7]]
    print 'peak intensity (V)', fitParameters[[2]], fitParameters[[2]] * fitParameters[[5]], \
         fitParameters[[2]] * fitParameters[[8]]
        
    dtSideModulationPeaks = np.amax( fitParameters[[0,3,6]] ) - np.amin( fitParameters[[0,3,6]] )
    dfOverDt = 12.0e9 / dtSideModulationPeaks    
    print 'df/dt = 12/{} GHz/s = {} THz/s'.format( \
        dtSideModulationPeaks, dfOverDt *1e-12 )
    cavityFwhmInS = transmissionFitParameters[1] * dfOverDt
    print 'cavity fwhm = {} s = {} GHz'.format( transmissionFitParameters[1], cavityFwhmInS*1e-9 )
    
    




## Example on how to use these functions (Plot and fit linewidth measurements Fri 23 June 2017)

In [ ]:
## LINEWIDTH IN GHz - DATA 23 June 2017

data_dir = '/Users/wjwesterveld/Documents/Temp/20170623/Scope' # copied above data to local temp folder, also to be sure we dont overwrite anyting


# Fri June 23th, 40 Hz sweeping speed
filename1 = os.path.join(data_dir,'IFM_OFF007.csv')
time1 = 0.005
time2 = 0.015
filename2 = os.path.join(data_dir,'IFM_OFF006.csv')
time3 = 0.005
time4 = 0.013
fwhmGuess = 0.001
timeDeltaGuess = 0.001
print '40 Hz sweeping speed, file1 ', os.path.basename(filename1), ', file2 ', os.path.basename(filename2)
fit_linewidth_GHz(filename1, time1, time2, filename2, time3, time4, fwhmGuess, timeDeltaGuess)

# fit   [ 0.00901127  0.0105146   0.00713837  ] for timeDelta = 0.003  
# fit   [ 0.00901126  0.01051459  0.00713838]   for timeDelta = 0.002
# fit   [ 0.00901127  0.0105146   0.00713837]   for timeDelta = 0.001



# Fri June 23th, 80 Hz sweeping speed
filename1 = os.path.join(data_dir,'IFM_OFF009.csv')
time1 = 0.000
time2 = 0.006
filename2 = os.path.join(data_dir,'IFM_OFF013.csv')
time3 = 0.000
time4 = 0.005
fwhmGuess = 0.002
timeDeltaGuess = 0.001
print ''
print ''
print '80 Hz sweeping speed, file1 ', os.path.basename(filename1), ', file2 ', os.path.basename(filename2)
fit_linewidth_GHz(filename1, time1, time2, filename2, time3, time4, fwhmGuess, timeDeltaGuess)

# Fri June 23th, 200 Hz sweeping speed
filename1 = os.path.join(data_dir,'IFM_OFF010.csv')
time1 = 0.001
time2 = 0.003
filename2 = os.path.join(data_dir,'IFM_OFF012.csv')
time3 = 0.001
time4 = 0.003
fwhmGuess = 0.001
timeDeltaGuess = 0.0001
print ''
print ''
print '200 Hz sweeping speed, file1 ', os.path.basename(filename1), ', file2 ', os.path.basename(filename2) 
# mpld3.enable_notebook().
fit_linewidth_GHz(filename1, time1, time2, filename2, time3, time4, fwhmGuess, timeDeltaGuess)
# mpld3.disable_notebook()

## Plot cavity transmittance measurements

In [ ]:
def tmp_plot_vibrations(data,time,time1,time2):
    timeResolution = time[1] - time[0]
    i1 = int( time1 / timeResolution )
    i2 = int( time2 / timeResolution )
    
    freq_pow, power = scipy.signal.welch(data[i1:i2], axis = 0, fs = 1/timeResolution,nperseg = i2-i1, detrend = 'linear' )
    j1 = np.argmax(freq_pow>0)
    j2 = np.argmax(freq_pow>200)        
    
    fig = plt.figure(figsize=(16, 8))

    plt.subplot(121)
    plt.plot( time[i1:i2], data[i1:i2] )   
    plt.xlabel('time (s)')
    plt.ylabel('signal (V)')  
        
    plt.subplot(122)    
    plt.plot(freq_pow[j1:j2], power[j1:j2])
    # plt.ylim([0,0.01])
    plt.xlabel('freq (Hz)')
    plt.ylabel('Noise power (V**2/Hz)')
    
    plt.show()  




In [ ]:
# Example of functions in previous cell


print 'red: vibrations'
dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170628/scope_DLM4038'
fileName = "IFM_OFF017.wdf.csv"  

timeWindow = 2       # s
fwhm = 8*1e9         # Hz
cavityLength = 15e-6 # m
centerFrequency = scipy.constants.c / 637.0e-9 # Hz

# time, data, timeCoarse, dataCoarse = load_drift_vibration_trace(dataDir, fileName)    
analyze_drift_vibration_trace(time, data, timeCoarse, dataCoarse, centerFrequency, fwhm, cavityLength, timeWindow )



## Simple analysis of vibration trace


In [ ]:
def load_simple_vibration_trace( dataDir, fileName ):
    data, timeResolution = read_scope_xviewer_csv( os.path.join(dataDir,fileName) )
    time = timeResolution * np.arange(data.shape[0]) 
    print 'time resolution {} us (sampling frequency {} kHz), trace length {} s, data length {}'.format( \
        timeResolution * 1e6, 1.0/timeResolution* 1e-3, timeResolution*len(data), data.shape[0] )
    freq_pow, power = scipy.signal.welch(data, axis = 0, fs = 1/timeResolution,nperseg = len(data), detrend = 'linear' )
    return data, time, freq_pow, power
    

## Measurements 31 8 2017: Photo-diode misaligned

In [ ]:
dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170831/scope'
fileName = "SCOPE_000.wdf.csv"  

data1, time, freq_pow, power1 =  load_simple_vibration_trace( dataDir, fileName )


In [ ]:
j1 = np.argmax(freq_pow>0)
j2 = np.argmax(freq_pow>60e3)    
fig = plt.figure(figsize=(16, 8))
plt.plot(freq_pow[j1:j2], 10.0*np.log10( power1[j1:j2] ) )
plt.grid()
plt.xlabel('freq (Hz)')
plt.ylabel('Noise power (V**2 in dB)')
plt.show()  

In [ ]:
j1 = np.argmax(freq_pow>0)
j2 = np.argmax(freq_pow>250)    
fig = plt.figure(figsize=(16, 8))
plt.plot(freq_pow[j1:j2], 10.0*np.log10( power1[j1:j2] ) )
plt.grid()
plt.xlabel('freq (Hz)')
plt.ylabel('Noise power (V**2 in dB)')
plt.show()  

In [ ]:
dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170831/scope'
fileName = "SCOPE_021.wdf.csv"  

data2, time, freq_pow, power2 =  load_simple_vibration_trace( dataDir, fileName )

In [ ]:
i1 = 0
i2 = 50000
fig = plt.figure(figsize=(16, 8))
plt.plot(time[i1:i2], data2[i1:i2] )
plt.grid()
plt.xlabel('time (s)')
plt.ylabel('V')
plt.show() 

In [ ]:
j1 = np.argmax(freq_pow>0)
j2 = np.argmax(freq_pow>6e3)    
fig = plt.figure(figsize=(16, 8))
plt.plot(freq_pow[j1:j2], 10.0*np.log10( power2[j1:j2] ) )
plt.grid()
plt.xlabel('freq (Hz)')
plt.ylabel('Noise power (V**2 in dB)')
plt.show()  

In [ ]:
j1 = np.argmax(freq_pow>0)
j2 = np.argmax(freq_pow>250)    
fig = plt.figure(figsize=(16, 8))
plt.plot(freq_pow[j1:j2], 10.0*np.log10( power2[j1:j2] ) )
plt.grid()
plt.xlabel('freq (Hz)')
plt.ylabel('Noise power (V**2 in dB)')
plt.show()  

In [ ]:
j1 = np.argmax(freq_pow>0)
j2 = np.argmax(freq_pow>50)    
fig = plt.figure(figsize=(16, 8))
plt.plot(freq_pow[j1:j2], 10.0*np.log10( power1[j1:j2] ), label='red' )
plt.plot(freq_pow[j1:j2], 10.0*np.log10( power2[j1:j2] ), label='green' )
plt.grid()
plt.xlabel('freq (Hz)')
plt.ylabel('Noise power (V**2 in dB)')
plt.legend()
plt.show()  

## Measurements 1 9 2017: Green

In [ ]:
dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170901/scope'
fileName = "SCOPE_000.wdf.csv"  

data, time, freq_pow, power =  load_simple_vibration_trace( dataDir, fileName )

In [ ]:
i1 = 0
i2 = 50000
fig = plt.figure(figsize=(16, 8))
plt.plot(time[i1:i2], data[i1:i2] )
plt.grid()
plt.xlabel('time (s)')
plt.ylabel('V')
plt.show() 

j1 = np.argmax(freq_pow>0)
j2 = np.argmax(freq_pow>250)    
fig = plt.figure(figsize=(16, 8))
plt.plot(freq_pow[j1:j2], 10.0*np.log10( power[j1:j2] ) )
plt.grid()
plt.xlabel('freq (Hz)')
plt.ylabel('Noise power (V**2 in dB)')
plt.show()  

In [ ]:
dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170901/scope'
fileName = "SCOPE_002.wdf.csv"  

data, time, freq_pow, power =  load_simple_vibration_trace( dataDir, fileName )



In [ ]:
fig = plt.figure(figsize=(16, 8))
plt.plot(time, data )
plt.grid()
plt.xlabel('time (s)')
plt.ylabel('V')
plt.show() 

i1 = np.argmax(time>10)
i2 = np.argmax(time>30)
d = data[i1:i2]

freq_pow, power = scipy.signal.welch(d, axis = 0, fs = 1/timeResolution,nperseg = len(d), detrend = 'linear' )

j1 = np.argmax(freq_pow>0)
j2 = np.argmax(freq_pow>250)    
fig = plt.figure(figsize=(16, 8))
plt.plot(freq_pow[j1:j2], 10.0*np.log10( power[j1:j2] ) )
plt.grid()
plt.xlabel('freq (Hz)')
plt.ylabel('Noise power (V**2 in dB)')
plt.show()  

In [ ]:
print i1, i2

## Measurements 1 9 2017: Red linewidth

In [ ]:
dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170901/scope'
fileName = "SCOPE_023.wdf.csv"  

data, timeResolution = read_scope_xviewer_csv( os.path.join(dataDir,fileName) )
time = timeResolution * np.arange(data.shape[0]) 
print 'time resolution {} us (sampling frequency {} kHz), trace length {} s, data length {}'.format( \
    timeResolution * 1e6, 1.0/timeResolution* 1e-3, timeResolution*len(data), data.shape[0] )



In [ ]:
timeStart  = 0.05595
timeWindow = 0.0008

i1 = np.argmax(time>timeStart)
i2 = np.argmax(time>timeStart+timeWindow)
d = data[i1:i2]
t = time[i1:i2]

fig = plt.figure(figsize=(16, 8))
plt.plot(t, d )
plt.grid()
plt.xlabel('time (s)')
plt.ylabel('V')
plt.show() 

In [ ]:
peakMinHeight = 0.35
peakMinWidth = 1

import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, filtfilt

def butter_lowpass(cutoff, fs, order=5):
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    b, a = butter(order, normal_cutoff, btype='low', analog=False)
    return b, a

def butter_lowpass_filtfilt(data, cutoff, fs, order=5):
    b, a = butter_lowpass(cutoff, fs, order=order)
    y = filtfilt(b, a, data)
    return y

dSmooth = butter_lowpass_filtfilt(data=d, cutoff=0.01/timeResolution, fs=1.0/timeResolution)


fig = plt.figure(figsize=(16, 8))
plt.plot( t, d )
plt.plot( t, dSmooth )
plt.grid()
plt.xlabel('time (s)')
plt.ylabel('V')
plt.show() 




In [ ]:
from scipy.signal import find_peaks_cwt

# i = find_peaks_cwt(d, np.arange(1, 550))

peakMin = 0.35
sideMin = 0.20

localMaximumMask = np.r_[True, dSmooth[1:] > dSmooth[:-1]] & np.r_[dSmooth[:-1] > dSmooth[1:], True]

peakMask = localMaximumMask & (dSmooth > peakMin)
sideMask = localMaximumMask & (dSmooth > sideMin) & (dSmooth < peakMin)

peakInd = np.where( peakMask )[0]
sideInd = np.where( sideMask )[0]

fig = plt.figure(figsize=(16, 8))
plt.plot( t, d )
plt.plot( t[peakInd], d[peakInd] , '+')
plt.plot( t[sideInd], d[sideInd] , 'x')
plt.grid()
plt.xlabel('time (s)')
plt.ylabel('V')
plt.show() 





In [ ]:
peakWin = 0.020e-3
peakWinEdgeMax = 0.10 # maximum value at edge of window
maxSidePeakAsymmetryHz = 1e9
fwhmGuess = 5e-6
timeDeltaGuess = 5e-6
sideModulationFrequency = 8.0e9

listFwhm = np.array([])

for i in peakInd:
    i1 = int( i - 0.5*peakWin/timeResolution )
    i2 = int( i + 0.5*peakWin/timeResolution )    
    dPeak = d[i1:i2]
    tPeak = t[i1:i2]

    fig = plt.figure(figsize=(16, 8))
    plt.plot( tPeak, dPeak )    
    plt.grid()
    plt.xlabel('time (s)')
    plt.ylabel('V')
    
    if (dPeak[0] > peakWinEdgeMax) | (dPeak[-1] > peakWinEdgeMax):
        print 'Signal at edge of window too high. Skipping.'
        plt.show()
        continue
    
    # Fit
    fitParameters, guessParameters = fit_lorenzian3(tPeak,dPeak,fwhmGuess,timeDeltaGuess,timeResolution)
    # Plot                    
    plt.plot( tPeak, lorentzian3(tPeak,guessParameters), linestyle=':' )
    plt.plot( tPeak, lorentzian3(tPeak,fitParameters) )
    plt.show()

    print 'fit'
    print 'peak positions (s)', fitParameters[[0,3,6]]
    print 'peak fwhm (s)', fitParameters[[1,4,7]]
    print 'peak intensity (V)', fitParameters[[2]], fitParameters[[2]] * fitParameters[[5]], \
         fitParameters[[2]] * fitParameters[[8]]

    dtSideModulationPeaks = np.amax( fitParameters[[0,3,6]] ) - np.amin( fitParameters[[0,3,6]] )
    dfOverDt = 2.0 * sideModulationFrequency / dtSideModulationPeaks    
    print 'df/dt = 2 * {}/{} GHz/s = {} THz/s'.format( sideModulationFrequency * 1e-9, dtSideModulationPeaks, dfOverDt *1e-12 )
    cavityFwhmInGhz = np.abs( fitParameters[1] * dfOverDt )
    print 'cavity fwhm = {} s = {} GHz'.format( fitParameters[1], cavityFwhmInGhz*1e-9 )    
    print 'sideband difference in GHz = {} GHz'.format( ( np.abs( fitParameters[3] - fitParameters[0] ) -  np.abs( fitParameters[6] - fitParameters[0] ) ) * dfOverDt *1e-9  )
    
    sidePeakAssymmetryHz = np.abs( ( np.abs( fitParameters[3] - fitParameters[0] ) -  np.abs( fitParameters[6] - fitParameters[0] ) ) ) * dfOverDt  
            
    if sidePeakAssymmetryHz > maxSidePeakAsymmetryHz:
        print 'Too large position assymetry between side-peaks. Skipping.'
        continue
    
    listFwhm = np.append( listFwhm, cavityFwhmInGhz )    
    

print 'number of peaks ', len(peakInd), ' ok fitted peaks ', len(listFwhm)
print 'mean fwhm GHz', np.mean( listFwhm ) *1e-9

## Measurements 1 9 2017: Red vibrations

In [ ]:
dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170901/scope'
fileName = "SCOPE_029.wdf.csv"  

data, timeResolution = read_scope_xviewer_csv( os.path.join(dataDir,fileName) )
time = timeResolution * np.arange(data.shape[0]) 
print 'time resolution {} us (sampling frequency {} kHz), trace length {} s, data length {}'.format( \
    timeResolution * 1e6, 1.0/timeResolution* 1e-3, timeResolution*len(data), data.shape[0] )







In [ ]:
fig = plt.figure(figsize=(16, 8))
plt.plot(time, data )
plt.grid()
plt.xlabel('time (s)')
plt.ylabel('V')
plt.show() 



In [ ]:
fig = plt.figure(figsize=(16, 8))
plt.hist(data,20)
plt.grid()
plt.show()


In [ ]:

freq_pow_tot, power_tot = scipy.signal.welch(data, axis = 0, fs = 1/timeResolution,nperseg = len(data), detrend = 'linear' )

In [ ]:
j1 = 0
j2 = np.argmax(freq_pow_tot>500)    
fig = plt.figure(figsize=(16, 8))
plt.plot( freq_pow_tot[j1:j2], 10.0*np.log10( power_tot[j1:j2] ) )
plt.grid( )
plt.xlabel('freq (Hz)')
plt.ylabel('Noise power (V**2 in dB)')
plt.show()  

print 'frequency resolution {} Hz, Nyquist {} MHz'.format( 1.0 / 1.0, 0.5 / timeResolution *1e-6 )

x1 = freq_pow_tot[j1:j2]
y1 = 10.0*np.log10( power_tot[j1:j2] )

In [ ]:
timeWindow = 0.032

print 'frequency resolution {} Hz, Nyquist {} MHz'.format( 1.0 / timeWindow, 0.5 / timeResolution *1e-6 )

# timeStart = 0.0
timeStart = 0.070
i1 = np.argmax(time>timeStart)
i2 = np.argmax(time>timeStart + timeWindow)
t = time[i1:i2]
d = data[i1:i2]


fig = plt.figure(figsize=(16, 8))
plt.plot(t, d )
plt.grid()
plt.xlabel('time (s)')
plt.ylabel('V')
plt.show() 


freq_pow, power = scipy.signal.welch(d, axis = 0, fs = 1/timeResolution,nperseg = len(d), detrend = 'linear' )

#j1 = np.argmax(freq_pow>0)
#j2 = np.argmax(freq_pow>31250)    
j1 = 0
j2 = len( d )
fig = plt.figure(figsize=(16, 8))
plt.plot(freq_pow[j1:j2]*1e-6, 10.0*np.log10( power[j1:j2] ) )
plt.grid()
plt.xlabel('freq (MHz)')
plt.ylabel('Noise power (V**2 in dB)')
plt.show()  




In [ ]:
freq_pow = freq_pow_tot
power = power_tot


freqPlotMax = 500

j1 = 0
j2 = np.argmax(freq_pow>freqPlotMax)    
fig = plt.figure(figsize=(12, 12))
plt.subplot(211)
plt.plot(freq_pow[j1:j2], 10.0*np.log10( power[j1:j2] ) )
plt.grid()
plt.xlabel('freq (Hz)')
plt.ylabel('Noise power (V**2 in dB)')



min_freq = 10
max_freq = freqPlotMax

min_ind = np.argmin(np.abs(freq_pow-min_freq))
max_ind = np.argmin(np.abs(freq_pow-max_freq))

pow_int = scipy.integrate.trapz(power[min_ind:max_ind],freq_pow[min_ind:max_ind],axis=0)
full_pow = scipy.integrate.trapz(power,freq_pow,axis=0)

# freq_rate = freq_pow[1]- freq_pow[0]

# ind_bin = int(freq_bin/freq_rate)
ind_bin = 1
min_ind = np.argmin(np.abs(freq_pow-min_freq))
max_ind = np.argmin(np.abs(freq_pow-max_freq))

cum_points = np.arange(min_ind,max_ind,ind_bin)
cum_freqs = freq_pow[cum_points[1:]]
cum_pow = np.zeros(np.shape(cum_freqs))

cum_freq = 0
for x in range(len(cum_pow)-1):
    cum_pow[x+1] = cum_pow[x] + scipy.integrate.trapz(power[cum_points[x]:cum_points[x+1]+1],freq_pow[cum_points[x]:cum_points[x+1]+1],axis=0)

cum_pow = cum_pow/(full_pow-pow_dc)

plt.subplot(212)
plt.plot(cum_freqs,cum_pow)
plt.grid()
plt.xlabel('freq (Hz)')
plt.ylabel('Integrated noise power (V**2)')
plt.show()

dc_freq = 10
dc_ind = np.argmin(np.abs(freq_pow-dc_freq))
pow_dc = scipy.integrate.trapz(power[0:dc_ind],freq_pow[0:dc_ind],axis=0)
rel_dc_pow = pow_dc/full_pow
print 'Relative DC power: %.3f' %rel_dc_pow
rel_pow = pow_int/full_pow
print 'Relative in-band power: %.3f' %rel_pow
rel_band_pow = pow_int/(full_pow-pow_dc)
print 'Relative in-band power (minus DC): %.3f' %rel_band_pow
print 'Full noise power: %.3e V/sqrt(Hz)' %np.sqrt(full_pow)

In [ ]:
j1 = 0
j2 = np.argmax(freq_pow>500)    
fig = plt.figure(figsize=(16, 8))
plt.plot( freq_pow[j1:j2], 10.0*np.log10( power[j1:j2] ) )
plt.grid()
plt.xlabel('freq (Hz)')
plt.ylabel('Noise power (V**2 in dB)')
plt.show()  


x2 = freq_pow[j1:j2]
y2 = 10.0*np.log10( power[j1:j2] )

In [ ]:
fig = plt.figure(figsize=(16, 8))
plt.plot( x1, y1, label='1s scan' )
plt.plot( x2, y2, label='quiet 32 ms' )
plt.plot( x3, y3, label='noisy 32 ms' )
plt.grid()
plt.xlabel('freq (Hz)')
plt.ylabel('Noise power (V**2 in dB)')
plt.legend()
plt.show()  

In [ ]:

freq_pow, power = scipy.signal.welch(data, axis = 0, fs = 1/timeResolution,nperseg = len(data), detrend = 'linear' )




In [ ]:
#j1 = np.argmax(freq_pow>0)
#j2 = np.argmax(freq_pow>31250)    
j1 = 0
j2 = np.argmax(freq_pow>250)    
fig = plt.figure(figsize=(16, 8))
plt.plot(freq_pow[j1:j2], 10.0*np.log10( power[j1:j2] ) )
plt.grid()
plt.xlabel('freq (Hz)')
plt.ylabel('Noise power (V**2 in dB)')
plt.show()  

print freq_pow[3] - freq_pow[2]

In [ ]:
peakTimes = np.array( [0.05596676, 0.0559952, 0.05602742, 0.05605493, 0.0560857, \
            0.05611793, 0.05614317, 0.056177, 0.05620576, 0.05623505 , 
            0.05626576, 0.05629522, 0.05635278] )

print 1.0 / np.mean( peakTimes[1:-1] - peakTimes[0:-2] )


In [ ]:
# Really ugly method to find full-width at fraction-of-max


lorentzFwhm = 5
lorentzFractionOfMax = 4.0 / 720.0

x = np.linspace( 0, 10*lorentzFwhm, 10000)
p = [0, lorentzFwhm, 1]
y = lorentzian(x,p)

lorentzFullWidthAtFractionOfMax = 2 * x[ np.argmin( y > lorentzFractionOfMax ) ]

plt.plot( x, y )
plt.plot( 0.5 * lorentzFullWidthAtFractionOfMax, lorentzian( 0.5 * lorentzFullWidthAtFractionOfMax, p ), '+' )
plt.grid()
plt.show()

print 'Lorenzian function with fwhm {}, full-width at {} of max = {}'.format( lorentzFwhm, lorentzFractionOfMax, lorentzFullWidthAtFractionOfMax )

In [ ]:
13.8 

## Measurements 5 9 2017

In [ ]:
dataDir = '/Users/wjwesterveld/Documents/Temp_CAV1_data/20170905/scope'
fileName = "SCOPE_029.wdf.csv"  

data, timeResolution = read_scope_xviewer_csv( os.path.join(dataDir,fileName) )
time = timeResolution * np.arange(data.shape[0]) 
print 'time resolution {} us (sampling frequency {} kHz), trace length {} s, data length {}'.format( \
    timeResolution * 1e6, 1.0/timeResolution* 1e-3, timeResolution*len(data), data.shape[0] )

In [ ]:
fig = plt.figure(figsize=(16, 8))
plt.plot(time, data )
plt.grid()
plt.xlabel('time (s)')
plt.ylabel('V')
plt.show() 

In [ ]:
timeWindow = 0.2

print 'frequency resolution {} Hz, Nyquist {} MHz'.format( 1.0 / timeWindow, 0.5 / timeResolution *1e-6 )

timeStart = 0.3
i1 = np.argmax(time>timeStart)
i2 = np.argmax(time>timeStart + timeWindow)
t = time[i1:i2]
d = data[i1:i2]

fig = plt.figure(figsize=(16, 8))
plt.plot(t, d )
plt.grid()
plt.xlabel('time (s)')
plt.ylabel('V')
plt.show() 

In [ ]:
fig = plt.figure(figsize=(16, 8))
plt.hist(data,20,label='1s')
plt.hist(d,20,label='window')
plt.legend()
plt.grid()
plt.show()



In [ ]:
from scipy.optimize import minimize
from scipy import interpolate

def lorentzian(x,p):    
    # Lorenzian function p[0] is center, p[1] is fwhm, p[2] is maximum intensity
    #    See http://mathworld.wolfram.com/LorentzianFunction.html
    #    But note that we not integrated intensity of one for p[2] = 1 but maxiumum intensity.
    return p[2] * (0.5 * p[1])**2 / ( ( x - p[0] )**2 + ( 0.5 * p[1] )**2 )


def lorentzFindX(lorentzFwhm, lorentzFractionOfMax):    
    def fun(x):
        return ( lorentzian( x, [0, lorentzFwhm, 1] ) -  lorentzFractionOfMax ) ** 2    
    res = minimize( fun, lorentzFwhm,  method='Nelder-Mead', bounds=( (0, None ), ) )
    if not( res.success ):
        print 'lorentzFindX convergence error: ', res.message       
    return res.x[0]
    
def lorentzFindFlankSteepness(lorentzFwhm, x):    
    return 8.0 * lorentzFwhm**2 * x / ( 4.0 * x**2 + lorentzFwhm**2 )**2
    
    
def analyzeTimeVibrations(d, lorentzFwhm, lorentzMaxV ):    
    meanVInD = np.mean( d ) 
    minVInD = np.amin(d)
    maxVInD = np.amax(d)
    # Compute vibrations in Hz using linear approximation of flank of lorentz
    flankSteepness = lorentzFindFlankSteepness( lorentzFwhm, lorentzFindX( lorentzFwhm, meanVInD / lorentzMaxV ) ) * lorentzMaxV
    vibrationsRMSV = np.sqrt(np.mean( (d - np.mean(d))**2))
    vibrationsRMSHzLinear = vibrationsRMSV / flankSteepness
    # Compute vibrations in Hz by first applying lorentzian function as lookup table V -> dHz
    minDFreqInD = lorentzFindX( lorentzFwhm, minVInD/lorentzMaxV )
    lorentzDFreq = np.linspace( -minDFreqInD-1e9, 0, 10000 )
    lorentzV = lorentzian(lorentzDFreq, [0, lorentzFwhm, lorentzMaxV] )
    dHz = interpolate.interp1d(lorentzV, lorentzDFreq)(d)    
    vibrationsRMSHzLorentz = np.sqrt( np.mean( ( dHz-np.mean(dHz) )**2) )
    # Print and plot results
    if True:   
        i1 = 0
        i2 = min( 500, len(d) )
        fig = plt.figure(figsize=(14, 6))
        plt.subplot(131)
        plt.plot( t[i1:i2], d[i1:i2] )
        plt.xlabel( 'time (s)' )
        plt.ylabel( 'transmission (V)' )
        plt.grid()
        plt.subplot(132)
        plt.plot( t[i1:i2], dHz[i1:i2]*1e-9 )
        plt.xlabel( 'time (s)' )
        plt.ylabel( 'cavity detuning (GHz)' )
        plt.grid()
        plt.subplot(133)
        plt.plot( lorentzDFreq*1e-9, lorentzV )
        plt.plot( -minDFreqInD*1e-9, minVInD, '+' )
        plt.plot( 0, lorentzMaxV, '+' )
        x0 = -lorentzFindX( lorentzFwhm, meanVInD / lorentzMaxV )*1e-9
        y0 = np.mean( d )
        plt.plot( [x0-20, x0+20], [y0-20e9*flankSteepness, y0+20e9*flankSteepness] )
        plt.xlabel( 'cavity detuning GHz' )
        plt.ylabel( 'V' )
        plt.grid()
        plt.show()
        print 'Assume Lorenzian linewidth with fwhm of {:.3f} GHz, transmission max {:.3f} V'.format( lorentzFwhm*1e-9, lorentzMaxV )
        print 'Trace min {:.3f} V, max {:.3f} V, mean {:.3f} V. Steepness of Lorentz flank at mean {} V/GHz'.format( minVInD, maxVInD, meanVInD, flankSteepness*1e9 )
        print 'rms vibrations {:.3f} V, {:.3f} GHz using linear approx, {:.3f} GHz using Lorenzian look-up'.format( vibrationsRMSV,  vibrationsRMSHzLinear*1e-9, vibrationsRMSHzLorentz*1e-9 )                        
    return vibrationsRMSV, vibrationsRMSHzLinear, vibrationsRMSHzLorentz
        

       
     
    
    
    
# Analysis
lorentzFwhm = 60e9
lorentzMaxV = np.amax( data )
cavLength = 3.3e-6
cavWavelength = 637e-9
cavFreq = scipy.constants.c / cavWavelength
vibrationsRMSV, vibrationsRMSHzLinear, vibrationsRMSHzLorentz = analyzeTimeVibrations(d, lorentzFwhm, lorentzMaxV )
vibrationsRMSmLinear = vibrationsRMSHzLinear * cavLength / cavFreq
vibrationsRMSmLorentz = vibrationsRMSHzLorentz * cavLength / cavFreq
print 'rms vibrations {:.3f} V, {:.3f} nm using linear approx, {:.3f} nm using Lorenzian look-up'.format( vibrationsRMSV,  vibrationsRMSmLinear*1e9, vibrationsRMSmLorentz*1e9 )                




In [ ]:
freq_pow, power = scipy.signal.welch(d, axis = 0, fs = 1/timeResolution,nperseg = len(d), detrend = 'linear' )





In [ ]:
freqPlotMax = 10e3

j1 = 0
j2 = np.argmax(freq_pow>freqPlotMax)    
fig = plt.figure(figsize=(12, 12))
plt.subplot(211)
plt.plot(freq_pow[j1:j2], 10.0*np.log10( power[j1:j2] ) )
plt.grid()
plt.xlabel('freq (Hz)')
plt.ylabel('Noise power (V**2 in dB)')

min_freq = 10
max_freq = freqPlotMax

min_ind = np.argmin(np.abs(freq_pow-min_freq))
max_ind = np.argmin(np.abs(freq_pow-max_freq))

pow_int = scipy.integrate.trapz(power[min_ind:max_ind],freq_pow[min_ind:max_ind],axis=0)
full_pow = scipy.integrate.trapz(power,freq_pow,axis=0)

# freq_rate = freq_pow[1]- freq_pow[0]

# ind_bin = int(freq_bin/freq_rate)
ind_bin = 1
min_ind = np.argmin(np.abs(freq_pow-min_freq))
max_ind = np.argmin(np.abs(freq_pow-max_freq))

cum_points = np.arange(min_ind,max_ind,ind_bin)
cum_freqs = freq_pow[cum_points[1:]]
cum_pow = np.zeros(np.shape(cum_freqs))

dc_freq = 10
dc_ind = np.argmin(np.abs(freq_pow-dc_freq))
pow_dc = scipy.integrate.trapz(power[0:dc_ind],freq_pow[0:dc_ind],axis=0)
rel_dc_pow = pow_dc/full_pow
print 'Relative DC power: %.3f' %rel_dc_pow
rel_pow = pow_int/full_pow
print 'Relative in-band power: %.3f' %rel_pow
rel_band_pow = pow_int/(full_pow-pow_dc)
print 'Relative in-band power (minus DC): %.3f' %rel_band_pow
print 'Full noise power: %.3e V/sqrt(Hz)' %np.sqrt(full_pow)


cum_freq = 0
for x in range(len(cum_pow)-1):
    cum_pow[x+1] = cum_pow[x] + scipy.integrate.trapz(power[cum_points[x]:cum_points[x+1]+1],freq_pow[cum_points[x]:cum_points[x+1]+1],axis=0)
        
cum_pow = cum_pow/(full_pow-pow_dc)

plt.subplot(212)
plt.plot(cum_freqs,cum_pow)
plt.grid()
plt.xlabel('freq (Hz)')
plt.ylabel('Integrated noise power (V**2)')
plt.show()

